In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
def get_habr_posts(query, pages):
    habr_blog = pd.DataFrame()
    for q in query:
        x = 1
        while x <= pages:

            URL = f'https://habr.com/ru/search/page{x}/'
            params = {'q': q}
            req = requests.get(URL, params=params)
            time.sleep(0.3)
            soup = BeautifulSoup(req.text)
            articles = soup.find_all('article', class_='tm-articles-list__item')

            for article in articles:
                if article.find('a', class_='tm-article-snippet__title-link'):
                    title = article.find('a', class_='tm-article-snippet__title-link').find('span').text
                else: title = article.find('h2', class_='tm-megapost-snippet__title').text

                if article.find('a', class_='tm-article-snippet__title-link'):
                    link = article.find('a', class_='tm-article-snippet__title-link').get('href')
                    link = 'https://habr.com' + link
                else:
                    link = article.find('header', class_='tm-megapost-snippet__header').find('a').get('href')
                    link = 'https://habr.com' + link

                req_2 = requests.get(link)
                soup_2 = BeautifulSoup(req_2.text)
                if soup_2.find('div', class_="article-formatted-body article-formatted-body article-formatted-body_version-1"):
                    text = soup_2.find('div', class_="article-formatted-body article-formatted-body article-formatted-body_version-1").text.strip()
                elif soup_2.find('div', class_ = 't-records'):
                    text = soup_2.find('div', class_ = 't-records').text.strip()
                else:
                    text = soup_2.find('p').text.strip()


                date = article.find('time').get('title')

                votes = article.find('svg', class_="tm-svg-img tm-votes-meter__icon tm-votes-meter__icon tm-votes-meter__icon_appearance-article").text
                votes = votes[17:]

                row = {'date': date, 'title': title, 'link': link, 'full text': text, 'likes' : votes}
                habr_blog = pd.concat([habr_blog, pd.DataFrame([row])])

            x = x+1
    return habr_blog.reset_index(drop=True)

In [ ]:
df = get_habr_posts(['python','анализ данных'], 1)

In [ ]:
df.drop_duplicates(inplace = True, subset=['link'])
df.sort_values(by=['date'], inplace = True)
df.reset_index(drop=True, inplace=True)
df.head(10)

,date,title,link,full text,likes
0,"2012-07-25, 13:29",Пять лет Школе анализа данных,https://habr.com/ru/company/yandex/blog/148443/,Ровно пять лет назад Яндекс объявил об открыти...,↑25 и ↓4
1,"2014-06-18, 15:19",Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/226641/,Данный выпуск дайджеста наиболее интересных ма...,↑22 и ↓3
2,"2014-06-30, 23:06",Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/228187/,Данный выпуск обзора наиболее интересных матер...,↑26 и ↓2
3,"2014-07-28, 13:10",Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/231323/,В очередном выпуске обзора наиболее интересных...,↑21 и ↓1
4,"2014-08-11, 10:47",Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/232879/,Представляю вашему вниманию очередной выпуск о...,↑23 и ↓0
5,"2014-09-14, 19:54",Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/236757/,Представляю вашему вниманию очередной выпуск о...,↑26 и ↓1
6,"2014-10-13, 14:00",Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/240139/,Представляю вашему вниманию очередной выпуск о...,↑20 и ↓1
7,"2014-11-23, 16:49",Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/243967/,Представляю вашему вниманию очередной выпуск о...,↑20 и ↓2
8,"2015-01-18, 18:02",Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/248165/,Представляю вашему вниманию очередной выпуск о...,↑17 и ↓0
9,"2017-06-27, 13:51",Использование Python и Excel для обработки и а...,https://habr.com/ru/company/otus/blog/331746/,Если Вы только начинаете свой путь знакомства ...,↑12 и ↓4
